In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from contextlib import redirect_stderr
from nltk import FreqDist
import pandas as pd
import numpy as np
import urllib
import time
import glob
import io
import re

In [ ]:
base_dir = 'D:\\data\\finance'

df_index = pd.read_csv(base_dir + '\\index_constituents.csv')
df_index_sp = df_index[df_index.conm == 'S&P 500 Comp-Ltd']
# df_sub = pd.read_csv(base_dir + '\\financial_statements\\us\\2019q4_notes\\sub.tsv', delimiter='\t')
# df_txt = pd.read_csv(base_dir + '\\financial_statements\\us\\2019q4_notes\\txt.tsv', delimiter='\t')
FORM_OF_INTEREST = ['10-Q', '10-K']

In [143]:
path = 'D:\\data\\finance\\financial_statements\\us\\EDGAR-raw'
df = pd.read_csv('D:\\data\\finance\\financial_statements\\us\\10k_2009_2019.csv', encoding='utf-8')
for idx, row in df.drop_duplicates().drop(['text', 'quarter'], axis=1).iterrows():
    url = 'https://www.sec.gov/Archives/edgar/data/' + str(row['cik']) + '/' + ''.join(row['adsh'].split('-')) + '/' + row['adsh'] + '.txt'
    urllib.request.urlretrieve(url,path + '\\' + row['adsh'] + '.txt')
    time.sleep(np.random.randint(100,400)/200)
    if idx % 100 == 0:
        print(row.filed)

2019-11-05 00:00:00
2019-12-05 00:00:00
2019-11-07 00:00:00
2019-11-08 00:00:00
2019-10-28 00:00:00
2019-10-31 00:00:00


In [16]:
def read_csv(file_path):
    with open(file_path, encoding = "ISO-8859-1") as f:
        columns = []
        lines_dicts = []
        for idx, line in enumerate(f):
            split = line.split('\n')[0].split('\t')
            if idx == 0:
                columns += split
            elif len(split) == 20:
                lines_dicts += [dict(zip(columns, split))]
            else:
                lines_dicts += [dict(zip(columns, split[:19] + [' '.join(split[19:])]))]
                
    return pd.DataFrame(lines_dicts)[['adsh', 'tag', 'value']]

In [4]:
df_i_want = pd.DataFrame(columns=['adsh', 'cik', 'gvkey', 'name', 'form', 'quarter', 'filed', 'text'])
df_i_want = df_i_want.set_index('adsh')
err = []
stderr_out = []
for yr in range(11):
    for q in range(4):
        file_dir = base_dir + '\\financial_statements\\us\\' + str(2009+yr) + 'q' + str(q+1) + '_notes\\'
        
        print(file_dir)
        df_sub = pd.read_csv(file_dir + 'sub.tsv', delimiter='\t', encoding = "ISO-8859-1")
        try:
            df_txt = pd.read_csv(file_dir + 'txt.tsv', delimiter='\t', encoding = "ISO-8859-1")[['adsh', 'tag', 'value']]
        except:
            df_txt = read_csv(file_dir + 'txt.tsv')[['adsh', 'tag', 'value']]
            
        df_sub = pd.merge(df_sub[df_sub.form.isin(FORM_OF_INTEREST)][['adsh', 'cik', 'filed', 'form']], \
                          df_index_sp[['gvkey', 'co_cik', 'co_conm', 'from', 'thru']], left_on='cik', right_on='co_cik', how='inner').drop('co_cik', axis=1)
        
        df_txt = df_txt[['adsh', 'tag', 'value']]
        df_txt = df_txt[(df_txt.tag.str.contains('TextBlock')) & (~df_txt.value.isnull())]
        
        df = pd.merge(df_txt, df_sub['adsh'], on='adsh')
        df_sub = df_sub[['adsh', 'cik', 'gvkey', 'co_conm', 'form', 'filed']].set_index('adsh')
        df_i_want = df_i_want.append(df_sub)
        for adsh in df_sub.index:
            try:
                df_i_want.loc[adsh, 'text'] = '. '.join(df[(df.adsh == adsh)].value)
            except:
                print(adsh)

D:\data\finance\financial_statements\us\2009q1_notes\
D:\data\finance\financial_statements\us\2009q2_notes\
D:\data\finance\financial_statements\us\2009q3_notes\
D:\data\finance\financial_statements\us\2009q4_notes\
D:\data\finance\financial_statements\us\2010q1_notes\
D:\data\finance\financial_statements\us\2010q2_notes\
D:\data\finance\financial_statements\us\2010q3_notes\
D:\data\finance\financial_statements\us\2010q4_notes\
D:\data\finance\financial_statements\us\2011q1_notes\
D:\data\finance\financial_statements\us\2011q2_notes\
D:\data\finance\financial_statements\us\2011q3_notes\
D:\data\finance\financial_statements\us\2011q4_notes\


C:\Users\sylim2357\anaconda3\envs\future-tense-mining\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


D:\data\finance\financial_statements\us\2012q1_notes\
D:\data\finance\financial_statements\us\2012q2_notes\
D:\data\finance\financial_statements\us\2012q3_notes\
D:\data\finance\financial_statements\us\2012q4_notes\
D:\data\finance\financial_statements\us\2013q1_notes\
D:\data\finance\financial_statements\us\2013q2_notes\
D:\data\finance\financial_statements\us\2013q3_notes\
D:\data\finance\financial_statements\us\2013q4_notes\
D:\data\finance\financial_statements\us\2014q1_notes\
D:\data\finance\financial_statements\us\2014q2_notes\
D:\data\finance\financial_statements\us\2014q3_notes\
D:\data\finance\financial_statements\us\2014q4_notes\
D:\data\finance\financial_statements\us\2015q1_notes\
D:\data\finance\financial_statements\us\2015q2_notes\
D:\data\finance\financial_statements\us\2015q3_notes\
D:\data\finance\financial_statements\us\2015q4_notes\
D:\data\finance\financial_statements\us\2016q1_notes\
D:\data\finance\financial_statements\us\2016q2_notes\
D:\data\finance\financial_st

In [5]:
df_i_want.to_csv('10k_2009_2019.csv', encoding='utf-8')